In [2]:
import os
import pandas as pd

In [29]:
import json

In [145]:
valfile = []
with open("/scratch/cse/btech/cs1200448/MatLlama/ift_cif_large/val.jsonl", 'r') as f:
    valfile = [json.loads(line) for line in f.readlines()]

out_dict = dict()
pred_dict = dict()

# tasks = ["atom count", "dimensions_synt", "atom name", "replace", "space group", "cell_volume", "formula", "dimensions_sem", "vol_calc"]

# ["infill", "formula_compute", "conditional_generation", "element_generation"]
task = 'element_generation'
tasks = ['element_generation'] 
for task in tasks:
    # out_dict[task] = []
    pred_dict[task] = []
    
for _, sample in enumerate(valfile):
    task = sample['task']
    system = sample['system']

    # with open(f'/home/cse/btech/cs1200448/MatLlama/cif_infer_outputs_cs/{_}.txt', 'r') as f:
    #     output = f.read()

    if task in tasks:
        # out_dict[task].append([output, sample])
        pred_dict[task].append([0, sample])
        
    # elif task=="dimensions":
    #     if "predict" not in system and "forecast" not in system:
    #         # out_dict["dimensions_synt"].append([output, sample])
    #         pred_dict["dimensions_synt"].append([output, sample])
    #     else:
    #         pred_dict["dimensions_sem"].append([output, sample])  

In [146]:
# for f in pred_dict['conditional_generation']:
#     print(f[1]['material'])

In [132]:
files = sorted(['15000_conditional_generation_llama3_1760_1925.csv',
 '15000_conditional_generation_llama3_0_1100.csv',
 '15000_conditional_generation_llama3_1100_1265.csv',
 '15000_conditional_generation_llama3_1430_1595.csv',
 '15000_conditional_generation_llama3_1265_1430.csv',
 '15000_conditional_generation_llama3_1595_1760.csv',
 '15000_conditional_generation_llama3_1925_2091.csv'])

df = pd.DataFrame()
for f in files:
    temp = pd.read_csv(f)
    df = pd.concat((df, temp))
df.shape


(2091, 4)

In [133]:
task = 'conditional_generation'

In [134]:
idxx = 0
pred_dict[task][idxx][1]['material']

'9eb1204082.CIF'

In [135]:
from tqdm import trange

In [136]:
task

'conditional_generation'

In [137]:
# idxx = 1
missed = []
for idxx in trange(len(df)):
    try:
        step_1 = df.pred_cs.iloc[idxx].split('Title')[0]
        length = step_1.split('\n')[0]
        angles = step_1.split('\n')[1]
        
        cif_file = 'data_global\n'
        cif_file += f"_chemical_name_mineral '{pred_dict[task][idxx][1]['material'][:-4]}'\n"
        
        # cif_file = ''
        
        for abc, lens in zip(['a','b','c'],length.strip().split()):
            line = f'_cell_length_{abc} {float(lens)}\n'
            cif_file += line
        
        for abc, ang in zip(['alpha','beta','gamma'],angles.strip().split()):
            line = f'_cell_angle_{abc} {float(ang)}\n'
            cif_file += line
        
        cif_file += 'loop_\n'
        cif_file += '_atom_site_label\n'
        cif_file += '_atom_site_fract_x\n'
        cif_file += '_atom_site_fract_y\n'
        cif_file += '_atom_site_fract_z\n'
        
        # Split the input data into lines and filter out empty lines
        lines = [line.strip() for line in step_1.splitlines()[2:] if line.strip()]
        
        # Initialize an empty list to hold the formatted lines
        formatted_lines = []
        
        # Iterate over the lines in pairs
        for i in range(0, len(lines), 2):
            element = lines[i]
            coordinates = lines[i + 1]
            formatted_lines.append(f"{element} {coordinates}")
            cif_file += f"{element} {coordinates}\n"
        
        # Write the formatted lines to a file
        output_filename = f"./{task}/{idxx}.CIF"

        if len(cif_file.strip().split('\n')[-1].split()) == 4:
            with open(output_filename, "w") as file:
                file.write(cif_file.strip())
        else:
            missed.append(idxx)
        # length, angle
    except:
        missed.append(idxx)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2091/2091 [00:02<00:00, 880.19it/s]


In [138]:
len(missed)

87

In [141]:
# print(df.pred_cs.iloc[missed[50]])

In [142]:
df1 = pd.read_csv('15000_element_generation_llama3_0_1045.csv')
df2 = pd.read_csv('15000_element_generation_llama3_1045_2091.csv')
df = pd.concat((df1, df2))

In [143]:
print(df.input.iloc[0])

Consider the following elements:
Ce
Nb
Mo
N
You need to create a stable crystal that contains at least one instance of each of these elements. It should not contain elements other than the specified ones. Generate a description of the lengths and angles of the lattice vectors and then the element type and coordinates for each atom within the lattice:
The output should be of the following format ONLY:
l1, l2, l3
a1, a2, a3
atom1
x, y, z
atom2
x, y, z
 ...

l1, l2, l3 should be the predicted cell lengths.
a1, a2, a3 should be the predicted cell angles.
atom1, atom2, atom3, and so on, should be replaced with atom names and corresponding x, y, z with their coordinates in the lattice.



In [149]:
print(df.pred_cs.iloc[1])

5.4 7.7 11.4
75 90 90
Gd
0.39 0.90 0.90
Gd
0.02 0.90 0.40
Tm
0.45 0.40 0.90
Tm
0.97 0.40 0.40
Tm
0.89 0.20 0.73
Tm
0.52 0.20 0.23
Lu
0.89 0.61 0.07
Lu
0.52 0.61 0.57
W
0.96 0.13 0.03
W
0.45 0.13 0.53
W
0.96 0.68 0.76
W
0.45 0.68 0.26
O
0.75 0.34 0.04
O
0.24 0.25 0.08
O
0.66 0.34 0.54
O
0.16 0.19 0.88
O
0.17 0.25 0.58
O
0.69 0.11 0.92
O
0.25 0.19 0.38
O
0.56 0.04 0.70
O
0.06 0.89 0.04
O
0.72 0.11 0.42
O
0.06 0.91 0.76
O
0.56 0.76 0.10
O
0.85 0.04 0.20
O
0.35 0.89 0.54
O
0.69 0.69 0.88
O
0.35 0.91 0.26
O
0.85 0.76 0.60
O
0.16 0.61 0.92
O
0.72 0.69 0.38
O
0.24 0.55 0.72
O
0.25 0.61 0.42
O
0.75 0.46 0.75
O
0.17 0.55 0.22
O
0.66 0.46 0.25Title: The effect of the 3.6 3.6 11.9 11.9 11.9. Abstract: The effect of the 3.6 3.6 11.9
11.9
: 90 90 120
Zn
0.39 0.26 0.74
Zn
0.39 0.26 0.06
Zn
0.39 0.26 0.40
Zn
0.72 0.92 0.07
Zn
0.05 0.59 0.73
Zn
0.39 0.92 0.24
Zn
0.39 0.92 0.57
Zn
0.39 0.92 0.90
Zn
0.72 0.92 0.74
Zn
0.72 0.92 0.40
Zn
0.05 0.59 0.07
Zn
0.05 0.59 0.40
Zn
0.72 0.59 0


In [150]:
task = 'element_generation'

In [151]:
missed = []
for idxx in trange(len(df)):
    try:
        step_1 = df.pred_cs.iloc[idxx].split('Title')[0]
        length = step_1.split('\n')[0]
        angles = step_1.split('\n')[1]
        
        cif_file = 'data_global\n'
        cif_file += f"_chemical_name_mineral '{pred_dict[task][idxx][1]['material'][:-4]}'\n"
        
        # cif_file = ''
        
        for abc, lens in zip(['a','b','c'],length.strip().split()):
            line = f'_cell_length_{abc} {float(lens)}\n'
            cif_file += line
        
        for abc, ang in zip(['alpha','beta','gamma'],angles.strip().split()):
            line = f'_cell_angle_{abc} {float(ang)}\n'
            cif_file += line
        
        cif_file += 'loop_\n'
        cif_file += '_atom_site_label\n'
        cif_file += '_atom_site_fract_x\n'
        cif_file += '_atom_site_fract_y\n'
        cif_file += '_atom_site_fract_z\n'
        
        # Split the input data into lines and filter out empty lines
        lines = [line.strip() for line in step_1.splitlines()[2:] if line.strip()]
        
        # Initialize an empty list to hold the formatted lines
        formatted_lines = []
        
        # Iterate over the lines in pairs
        for i in range(0, len(lines), 2):
            element = lines[i]
            coordinates = lines[i + 1]
            formatted_lines.append(f"{element} {coordinates}")
            cif_file += f"{element} {coordinates}\n"
        
        # Write the formatted lines to a file
        output_filename = f"./{task}/{idxx}.CIF"

        if len(cif_file.strip().split('\n')[-1].split()) == 4:
            with open(output_filename, "w") as file:
                file.write(cif_file.strip())
        else:
            missed.append(idxx)
        # length, angle
    except:
        missed.append(idxx)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2091/2091 [00:02<00:00, 898.20it/s]


In [152]:
len(missed)

52

In [1]:
2091-52

2039